In [2]:
#  %pip install sentence-transformers

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


d:\PycharmProjects\Resume-Scoring-main\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [6]:
import pandas as pd
# this csv contains cleaned data by preprocessing it manually using regex
df_jds = pd.read_csv('jds.csv')
df_jds.head()


,company_name,position_title,skills
0,Grand-flo Spritvest Sdn Bhd,Software Engineer (Web),develop test and implement new or existing sof...
1,Legal Marketing And Staffing,Information Technology System Administrator,maintain network integrity and security
2,Randstad,Office Clerk,filing
3,Cardinal Logistics,CDL-A Truck Driver - $5K Sign On Bonus,required skills class a commercial driver s li...
4,Motivate LLC,Assistant Manager,onboarding


In [7]:
df_res = pd.read_csv('parsed_res_it.csv')
df = df_res.sample(frac=1, random_state=42)
df_res = df_res[1:22]
df_res.drop(7)

,Unnamed: 0,Category,Skills,Education,Experience
1,1,INFORMATION TECHNOLOGY,"skills for networking issues, end user problem...",education programs to explain upgrades to end ...,"experienced in server management, systems anal..."
2,2,INFORMATION TECHNOLOGY,skills to interface effectively with all level...,education master of science : leadership walde...,experience 04/2013 to current information tech...
3,3,WORKING RF,"skills big data, c, c++, charts, circuit desig...",education m.s : electrical and computer engine...,experience working rf systems engineer may 20...
4,4,INFORMATION TECHNOLOGY,"skills active directory, administrative suppor...","education computer technician , 07/2006 to 06/...","experience information technology manager , 03..."
5,5,IT MANAGEMENT,skills. (solar),education and certificates troubleshooting q c...,experience in hardware and software troublesho...
6,6,INFORMATION TECHNOLOGY,"skills desktops, ethernet cables cisco routers...",education and certificates troubleshooting q c...,experience with information technology service...
8,8,INFORMATION TECHNOLOGY,"skills, strong problem solving skills. sound w...",education and training 1 1988 curry college c...,experience. strengths - excellent communication
9,9,INFORMATION TECHNOLOGY,skills for quality development of standard doc...,education master of science : computer & infor...,experience to deliver insightful network infra...
10,10,TECHNOLOGY SUPPORT,skills applications: ms office suite 2000-2016...,education department located on the main campu...,experience 03/2016 to current lead information...
11,11,INFORMATION TECHNOLOGY,"skills administrative, analyst, army, agency, ...",education bachelor of science (bs) : informati...,experience information technology (it) special...


In [8]:
sentences_res = df_res["Skills"].to_list()

for s in sentences_res:
    if type(s) != str:
        sentences_res.remove(s)#removing 1 particular nan value

sentences_res = sentences_res[:20]
sentences_res

['skills for networking issues, end user problems, and network security. ',
 'skills to interface effectively with all levels of an organization. expertise spans: network engineering, helpdesk administration, software licensing, disaster recovery, backup/restore operations/project management, strategic planning/analysis, budgeting, teambuilding/training, vendor relations policy/procedure development, quality assurance, troubleshooting, problem solving, process improvement. ',
 'skills big data, c, c++, charts, circuit design, hardware, data acquisition, data analyst, data collection, data mining, databases, database, dbm, dti, design software, documentation, functional, gsm, innovation, java, labview, team leader, logic analyzer, mac, manufacturing processes, matlab, excel, microsoft office, office, microwave, radar, ncs, network, db, packaging, pivot tables, programming, project design, proposals, publication, python, quality, requirement, research, sas, self-starter, spectrum analyze

In [9]:
sentences_jds = df_jds["skills"].to_list()

for s in sentences_jds:
    if type(s) != str:
        sentences_jds.remove(s)  # removing 1 particular nan value

sentences_jds


['develop test and implement new or existing software products and continuously looking for improvements',
 'maintain network integrity and security',
 'filing',
 'required skills class a commercial driver s license cdla',
 'onboarding',
 'required skills must be currently enrolled in college and pursuing coursework in marketing or communication ability to develop and maintain effective working relationships with stakeholders including employees and the general public ability to speak and write clearly',
 'returns and sales answer and resolve calls from sales colleagues and customers regarding guidelines increases and credit referrals complete various credit customer service functions based on business needs',
 'prepare and submit customer applications to insurance carriers for approval review and compile necessary information ensuring application packet completeness and accuracy verify proper formatting and submit the application using the carrier specified channel',
 'provide full se

In [10]:
example_sentence = sentences_res[0]
example_tokens = tokenizer.encode_plus(example_sentence, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
example_tokens


{'input_ids': tensor([[    0,  4817,  2009, 14052,  3318,  1014,  2207,  5314,  3475,  1014,
          2002,  2901,  3040,  1016,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [11]:
import torch




def tokenize_sentence(sentences):
    tokens = {'input_ids': [], 'attention_mask': []}
    for sentence in sentences:
        try:
            new_tokens = tokenizer.encode_plus(
                sentence, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
            tokens['input_ids'].append(new_tokens['input_ids'][0])
            tokens['attention_mask'].append(new_tokens['attention_mask'][0])
        except Exception as e:
            print(f"Error processing sentence: {sentence}")
            print(f"Error message: {str(e)}")

    # Convert the list of tensors into a tuple
    input_ids_tuple = tuple(tokens['input_ids'])
    attention_mask_tuple = tuple(tokens['attention_mask'])

    # Use torch.stack with the tuples of tensors
    tokens['input_ids'] = torch.stack(input_ids_tuple)
    tokens['attention_mask'] = torch.stack(attention_mask_tuple)
    return tokens



In [12]:
tokens_res = tokenize_sentence(sentences_res)
tokens_jds = tokenize_sentence(sentences_jds)


In [13]:
print(f'''{tokens_res['input_ids'].shape}
{tokens_jds['input_ids'].shape}''')


torch.Size([20, 512])
torch.Size([15, 512])


: 

In [14]:
outputs_res = model(**tokens_res)

outputs_jds = model(**tokens_jds)
print(outputs_res, outputs_jds)


In [ ]:
outputs_res.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
embeddings_res = outputs_res.last_hidden_state
embeddings_jds = outputs_jds.last_hidden_state
print(embeddings_res, embeddings_jds)

tensor([[[ 0.0638, -0.0223, -0.0726,  ..., -0.0891,  0.1923, -0.0530],
         [-0.0542, -0.1651, -0.1084,  ...,  0.1009,  0.1772, -0.0542],
         [ 0.0029, -0.0241, -0.0135,  ...,  0.0113,  0.1817, -0.0429],
         ...,
         [-0.0479,  0.0616, -0.1104,  ..., -0.0370,  0.2172, -0.0496],
         [-0.0479,  0.0616, -0.1104,  ..., -0.0370,  0.2172, -0.0496],
         [-0.0479,  0.0616, -0.1104,  ..., -0.0370,  0.2172, -0.0496]],

        [[-0.0116, -0.0754, -0.0156,  ...,  0.0159,  0.0869, -0.1285],
         [-0.0295, -0.0597, -0.0788,  ...,  0.1064,  0.0005, -0.1817],
         [ 0.0358, -0.0873, -0.1843,  ...,  0.0783,  0.1769, -0.0796],
         ...,
         [-0.0464,  0.1390, -0.0329,  ...,  0.0948,  0.1409, -0.0754],
         [-0.0464,  0.1390, -0.0329,  ...,  0.0948,  0.1409, -0.0754],
         [-0.0464,  0.1390, -0.0329,  ...,  0.0948,  0.1409, -0.0754]],

        [[ 0.0176, -0.0452, -0.0186,  ...,  0.0690,  0.1560, -0.0189],
         [ 0.0570, -0.0504, -0.0360,  ...,  0

In [ ]:
embeddings_res.shape

torch.Size([20, 512, 768])

In [ ]:
attention_mask_res = tokens_res['attention_mask']
attention_mask_jds = tokens_jds['attention_mask']

In [ ]:
resized_attention_mask_res = attention_mask_res.unsqueeze(-1).expand(embeddings_res.size()).float()
resized_attention_mask_jds = attention_mask_jds.unsqueeze(-1).expand(embeddings_jds.size()).float()


In [ ]:
print(resized_attention_mask_res, resized_attention_mask_jds)

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [ ]:
resized_attention_mask_jds[0][0].shape

torch.Size([768])

In [ ]:
masked_embedding_res = embeddings_res * resized_attention_mask_res
masked_embedding_res.shape
masked_embedding_jds = embeddings_jds * resized_attention_mask_jds
masked_embedding_jds.shape


torch.Size([15, 512, 768])

In [ ]:
print(masked_embedding_res, masked_embedding_jds)

tensor([[[ 0.0638, -0.0223, -0.0726,  ..., -0.0891,  0.1923, -0.0530],
         [-0.0542, -0.1651, -0.1084,  ...,  0.1009,  0.1772, -0.0542],
         [ 0.0029, -0.0241, -0.0135,  ...,  0.0113,  0.1817, -0.0429],
         ...,
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000]],

        [[-0.0116, -0.0754, -0.0156,  ...,  0.0159,  0.0869, -0.1285],
         [-0.0295, -0.0597, -0.0788,  ...,  0.1064,  0.0005, -0.1817],
         [ 0.0358, -0.0873, -0.1843,  ...,  0.0783,  0.1769, -0.0796],
         ...,
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000]],

        [[ 0.0176, -0.0452, -0.0186,  ...,  0.0690,  0.1560, -0.0189],
         [ 0.0570, -0.0504, -0.0360,  ...,  0

In [ ]:
summed_masked_embeddings_res = torch.sum(masked_embedding_res, 1)
summed_masked_embeddings_jds = torch.sum(masked_embedding_jds, 1)



In [ ]:
print(summed_masked_embeddings_res, summed_masked_embeddings_jds)

tensor([[-0.0214, -0.5595, -0.8901,  ..., -0.7069,  2.4902, -1.1125],
        [-1.2205, -0.9466, -0.7464,  ...,  1.2782,  2.6587, -2.6356],
        [-0.6054, -4.6724, -0.7509,  ...,  2.5821,  6.6477, -1.3908],
        ...,
        [-5.5199,  5.3130, -6.2001,  ...,  7.1198, 12.2640, -2.7251],
        [ 2.4542, -8.7723, -9.1892,  ...,  1.3914, 17.5477,  0.5174],
        [ 0.4720,  0.3210, -0.4807,  ...,  0.7508,  1.2484, -1.0538]],
       grad_fn=<SumBackward1>) tensor([[ 3.6319, -0.5037, -2.7960,  ..., -1.4151,  1.9501, -2.5900],
        [-0.7370,  2.3888,  0.8173,  ...,  1.2526,  1.0763, -0.4517],
        [ 0.0096, -0.1501,  0.0660,  ..., -0.1841, -0.0092, -0.1233],
        ...,
        [ 2.7517, -0.0103, -2.3426,  ..., -0.4373, -0.8095, -1.2891],
        [ 0.2834,  0.1424, -0.0172,  ..., -0.4015, -0.3233,  0.0491],
        [ 0.6103,  0.3027, -0.5265,  ..., -0.1255, -0.2689, -0.3328]],
       grad_fn=<SumBackward1>)


In [ ]:
resized_attention_mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [ ]:
count_of_one_in_mask_tensor_res = torch.clamp(resized_attention_mask_res.sum(1), min=1e-9 )
count_of_one_in_mask_tensor_jds = torch.clamp(resized_attention_mask_jds.sum(1), min=1e-9)

In [ ]:
print(count_of_one_in_mask_tensor_res,count_of_one_in_mask_tensor_jds)

tensor([[ 15.,  15.,  15.,  ...,  15.,  15.,  15.],
        [ 24.,  24.,  24.,  ...,  24.,  24.,  24.],
        [ 35.,  35.,  35.,  ...,  35.,  35.,  35.],
        ...,
        [116., 116., 116.,  ..., 116., 116., 116.],
        [ 76.,  76.,  76.,  ...,  76.,  76.,  76.],
        [  8.,   8.,   8.,  ...,   8.,   8.,   8.]]) tensor([[16., 16., 16.,  ..., 16., 16., 16.],
        [ 7.,  7.,  7.,  ...,  7.,  7.,  7.],
        [ 3.,  3.,  3.,  ...,  3.,  3.,  3.],
        ...,
        [22., 22., 22.,  ..., 22., 22., 22.],
        [ 4.,  4.,  4.,  ...,  4.,  4.,  4.],
        [ 4.,  4.,  4.,  ...,  4.,  4.,  4.]])


In [ ]:
mean_pooled_res = summed_masked_embeddings_res / count_of_one_in_mask_tensor_res
mean_pooled_jds = summed_masked_embeddings_jds / count_of_one_in_mask_tensor_jds


In [ ]:
mean_pooled_jds.shape

torch.Size([15, 768])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

mean_pooled_res = mean_pooled_res.detach().numpy()
mean_pooled_jds = mean_pooled_jds.detach().numpy()


In [ ]:
res = cosine_similarity([mean_pooled_jds[0]], mean_pooled_res[0:])
res

array([[0.2735528 , 0.35187927, 0.31203234, 0.19061683, 0.24113005,
        0.2887963 , 0.300919  , 0.29175347, 0.2041206 , 0.3292647 ,
        0.2919261 , 0.26974857, 0.2553419 , 0.26320016, 0.31945303,
        0.34041438, 0.21112108, 0.288772  , 0.34612405, 0.24107727]],
      dtype=float32)

In [ ]:
print(f"Job: {df_jds['position_title'][0]}")
categs = df_res['Category']
data = {'Category': categs[1:], 'Scores': res[0]}
df_scores = pd.DataFrame(data)
df_scores = df_scores.sort_values(by='Scores', ascending=False)
df_scores[:5]

Job: Software Engineer (Web)


,Category,Scores
3,Professional Summary,0.351879
20,general ledger,0.346124
17,SUPERVISOR ACCOUNTANT,0.340414
11,PROJECT ACCOUNTANT,0.329265
16,ACCOUNTANT III,0.319453
